In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rioxarray
import xarray
from rioxarray.merge import merge_arrays

In [ ]:
cont = 'na' # af,as,au,na,sa,eu
lvl = '03'
basinID = 732

In [ ]:
bucket_name = 'opera-pst-rs-pop1'
local_path = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr/"

In [ ]:
with open('dswx_manifest.txt') as f:
    lines = f.readlines()

print(lines[0])

In [ ]:
keys = []
for line in lines:
    prefix = line.split(bucket_name+'/')[-1][:-2]
    filename = prefix.split('/')[-1]+'_B01_WTR.tiff'
    keys.append(filename)
print(keys[0])

In [ ]:
basin_path  = f'./shapefiles/basins/hybas_{cont}/hybas_{cont}_lev{lvl}_v1c.shp'
basin_gdf = gpd.read_file(basin_path)
basin_gdf.head()

In [ ]:
mrg_gdf = gpd.read_file('./shapefiles/world_mgrs/mgrs_region.shp') 
mrg_gdf.head()

In [ ]:
target_basin =  basin_gdf[basin_gdf.PFAF_ID==basinID]
target_basin.plot()

In [ ]:
grid_codes = gpd.sjoin(mrg_gdf,target_basin)
grid_codes.plot()

In [ ]:
tile_list = []
for row in grid_codes.iterrows():
    tile_list.append(row[1].iloc[0]+row[1].iloc[1])
len(tile_list)

In [ ]:
keys = []
for line in lines:
    prefix = line.split(bucket_name+'/')[-1][:-2]
    filename = prefix.split('/')[-1]+'_B01_WTR.tiff'
    tile = filename.split('_')[4][1:]
    if tile in tile_list:
        keys.append(filename)
        #tile_list.remove(tile)
print(keys[0])
print(len(keys))

In [ ]:
ds_to_mosaic = []
i = 0
for key in keys:
    file_path = f'{local_path}{key}'
    #xds = rioxarray.open_rasterio(file_path,chunks={'x':'auto','y':'auto'})
    xds = rioxarray.open_rasterio(file_path,cache=False)
    reproj = xds.rio.reproject("EPSG:4326")
    ds_to_mosaic.append(reproj)
    del xds
    del  reproj
    if i%100 == 0:
        print(f'Finshed loading {i} tiles')
    i = i+1
ds_to_mosaic[0]

In [ ]:
len(ds_to_mosaic)

In [ ]:
merged_raster =  merge_arrays(dataarrays = ds_to_mosaic, crs="EPSG:4326", nodata = 255)
#merged_raster.plot()

In [ ]:
merged_raster.rio.to_raster(f'mosaic/{cont}_basin{basinID}_WTR.tif', tiled=True, windowed=True)